In [1]:
!pip install transformers==4.17
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 44.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.17.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.9 MB/s eta 0:00:00


In [2]:
from transformers import Trainer, TrainingArguments, DistilBertTokenizer, DistilBertForSequenceClassification
import torch
import pandas as pd
import nltk
import re
import ast
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split
import torch.nn as nn
import optuna
import time

In [3]:
splits = {'pharo_train': 'data/pharo_train-00000-of-00001.parquet', 'pharo_test': 'data/pharo_test-00000-of-00001.parquet'}
train_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["pharo_train"])
test_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["pharo_test"])

display(train_df.head())
display(test_df.head())
display(train_df.shape)
display(test_df.shape)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,index,class,comment_sentence,partition,combo,labels
0,0,BlAnchorRelativeToElement,relative anchor takes an arbitrary element as ...,0,relative anchor takes an arbitrary element as ...,"[0, 0, 1, 0, 0, 0, 0]"
1,2,BlArrowheadSimpleArrow,i am a simple arrow like arrowhead.,0,i am a simple arrow like arrowhead. | BlArrowh...,"[0, 0, 0, 0, 1, 0, 0]"
2,3,BlArrowheadSimpleArrow,both my size and arrow length depend on the cu...,0,both my size and arrow length depend on the cu...,"[0, 0, 0, 0, 1, 0, 0]"
3,4,BlArrowheadSimpleArrow,it is possible hovewer to customize a length f...,0,it is possible hovewer to customize a length f...,"[0, 0, 0, 0, 0, 1, 0]"
4,8,BlArrowheadTriangle,i am a triangular arrow head.,0,i am a triangular arrow head. | BlArrowheadTri...,"[0, 0, 0, 0, 1, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,1,BlArrowheadExamples,i contain examples of different types of blarr...,1,i contain examples of different types of blarr...,"[0, 0, 1, 1, 0, 0, 0]"
1,5,BlArrowheadSimpleArrow,users can also customise nose angle that tells...,1,users can also customise nose angle that tells...,"[0, 1, 0, 0, 0, 0, 0]"
2,6,BlArrowheadSimpleArrow,the with of the outer arrows can be specified ...,1,the with of the outer arrows can be specified ...,"[0, 0, 1, 0, 0, 0, 0]"
3,7,BlArrowheadSimpleArrow,i support both background and border paint and...,1,i support both background and border paint and...,"[0, 0, 1, 0, 0, 0, 0]"
4,9,BlArrowheadTriangle,my size depends on the width of a curve and he...,1,my size depends on the width of a curve and he...,"[0, 0, 1, 0, 0, 0, 0]"


(1298, 6)

(289, 6)

In [4]:
def remove_punctuation_except_pipe(text):
    return re.sub(r"[^\w\s\|]", "", text)

# Apply the function to the 'combo' column
train_df['combo'] = train_df['combo'].apply(remove_punctuation_except_pipe)
test_df['combo'] = test_df['combo'].apply(remove_punctuation_except_pipe)

# Convert data to lowercase
train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)
test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)

train_df[["class", "comment_sentence", "combo"]] = train_str_df[["class", "comment_sentence", "combo"]]
test_df[["class", "comment_sentence", "combo"]] = test_str_df[["class", "comment_sentence", "combo"]]

display(train_df.head())
display(test_df.head())

<ipython-input-4-df4392ae2e99>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
<ipython-input-4-df4392ae2e99>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(


,index,class,comment_sentence,partition,combo,labels
0,0,blanchorrelativetoelement,relative anchor takes an arbitrary element as ...,0,relative anchor takes an arbitrary element as ...,"[0, 0, 1, 0, 0, 0, 0]"
1,2,blarrowheadsimplearrow,i am a simple arrow like arrowhead.,0,i am a simple arrow like arrowhead | blarrowhe...,"[0, 0, 0, 0, 1, 0, 0]"
2,3,blarrowheadsimplearrow,both my size and arrow length depend on the cu...,0,both my size and arrow length depend on the cu...,"[0, 0, 0, 0, 1, 0, 0]"
3,4,blarrowheadsimplearrow,it is possible hovewer to customize a length f...,0,it is possible hovewer to customize a length f...,"[0, 0, 0, 0, 0, 1, 0]"
4,8,blarrowheadtriangle,i am a triangular arrow head.,0,i am a triangular arrow head | blarrowheadtria...,"[0, 0, 0, 0, 1, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,1,blarrowheadexamples,i contain examples of different types of blarr...,1,i contain examples of different types of blarr...,"[0, 0, 1, 1, 0, 0, 0]"
1,5,blarrowheadsimplearrow,users can also customise nose angle that tells...,1,users can also customise nose angle that tells...,"[0, 1, 0, 0, 0, 0, 0]"
2,6,blarrowheadsimplearrow,the with of the outer arrows can be specified ...,1,the with of the outer arrows can be specified ...,"[0, 0, 1, 0, 0, 0, 0]"
3,7,blarrowheadsimplearrow,i support both background and border paint and...,1,i support both background and border paint and...,"[0, 0, 1, 0, 0, 0, 0]"
4,9,blarrowheadtriangle,my size depends on the width of a curve and he...,1,my size depends on the width of a curve and he...,"[0, 0, 1, 0, 0, 0, 0]"


In [5]:
X = list(train_df['combo'])
y = list(train_df['labels'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [6]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=7)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map

In [7]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, device="cpu"):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.device = device

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoded_text = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=256,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoded_text['input_ids'].squeeze().to(self.device)
        attention_mask = encoded_text['attention_mask'].squeeze().to(self.device)
        label = torch.tensor(label, dtype=torch.float).to(self.device)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label
        }

train_dataset = TextClassificationDataset(X_train, y_train, tokenizer)
eval_dataset = TextClassificationDataset(X_val, y_val, tokenizer)

In [8]:
class OneHotTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Use BCEWithLogitsLoss for multi-label classification
        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits, labels.float())

        return (loss, outputs) if return_outputs else loss

In [9]:
def compute_metrics(pred):
    logits = pred.predictions
    preds = (logits > 0).astype(int)

    labels = pred.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = (preds == labels).mean()

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


# Optional compute_metrics function to get the accuracy, precision, recall, f1 for each of the individual labels
# without averaging them

# from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# def compute_metrics(pred):
#     # Get predictions and apply sigmoid to get probabilities
#     logits = pred.predictions
#     preds = (logits > 0).astype(int)  # Threshold at 0 to get binary predictions

#     # Get true labels
#     labels = pred.label_ids

#     # Calculate precision, recall, and f1-score for each label without averaging
#     precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average=None)
#     # Calculate accuracy per label (how many times each label is correctly predicted)
#     accuracy_per_label = (preds == labels).mean(axis=0)

#     # Prepare a dictionary to store metrics per category
#     metrics = {
#         'accuracy_per_label': accuracy_per_label,
#         'precision_per_label': precision,
#         'recall_per_label': recall,
#         'f1_per_label': f1
#     }

#     # Optionally, if you want to calculate an overall accuracy:
#     overall_accuracy = (preds == labels).all(axis=1).mean()  # This gives the fraction of samples where all labels are correctly predicted
#     metrics['overall_accuracy'] = overall_accuracy

#     return metrics

In [10]:
def objective(trial):
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
    num_train_epochs = trial.suggest_int("num_train_epochs", 3, 10)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)

    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size * 2,
        warmup_steps=10,
        weight_decay=weight_decay,
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        gradient_accumulation_steps=2,
        report_to="none",
    )

    trainer = OneHotTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    eval_results = trainer.evaluate()

    return eval_results["eval_f1"]

In [11]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

print("Best trial:")
print(study.best_trial)

[I 2024-12-01 14:16:24,287] A new study created in memory with name: no-name-ed166ca4-021f-4b97-bc85-d6904881f17d
<ipython-input-10-9265609bdcc0>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-9265609bdcc0>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=Tru

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.595593,0.842308,0.128205,0.003497,0.006807
2,No log,0.497800,0.842857,0.000000,0.000000,0.000000
3,No log,0.453009,0.842857,0.000000,0.000000,0.000000
4,No log,0.430244,0.842857,0.000000,0.000000,0.000000
5,No log,0.417431,0.842857,0.000000,0.000000,0.000000
6,No log,0.409395,0.843407,0.384615,0.003497,0.006930
7,No log,0.404177,0.852747,0.384615,0.062937,0.108173
8,0.480500,0.400715,0.850549,0.384615,0.048951,0.086849
9,0.480500,0.398681,0.855495,0.384615,0.080420,0.133025
10,0.480500,0.398012,0.858242,0.384615,0.097902,0.156076


***** Running Evaluation *****
  Num examples = 260
  Batch size = 16
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
***** Running Evaluation *****
  Num examples = 260
  Batch size = 16
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
***** Running Evaluation *****
  Num examples = 260
  Batch size = 16
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in la

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2024-12-01 14:20:49,832] Trial 0 finished with value: 0.1560758082497213 and parameters: {'learning_rate': 1.6988743069743781e-06, 'batch_size': 8, 'num_train_epochs': 10, 'weight_decay': 0.0035024059700821735}. Best is trial 0 with value: 0.1560758082497213.
<ipython-input-10-9265609bdcc0>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-9265609bdcc0>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This fea

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.384205,0.875824,0.361991,0.223776,0.276577
2,No log,0.372417,0.871429,0.303419,0.248252,0.273077
3,No log,0.362482,0.871978,0.289377,0.276224,0.282648
4,0.383900,0.354714,0.873626,0.295858,0.279720,0.287563
5,0.383900,0.348289,0.875275,0.298347,0.290210,0.294222
6,0.383900,0.343525,0.875824,0.303254,0.286713,0.294752
7,0.383900,0.339722,0.876923,0.302707,0.297203,0.299929
8,0.351700,0.337180,0.876923,0.302707,0.297203,0.299929
9,0.351700,0.335658,0.877473,0.303458,0.300699,0.302072
10,0.351700,0.335187,0.877473,0.303458,0.300699,0.302072


...

[I 2024-12-01 15:17:37,127] Trial 18 finished with value: 0.795601985136869 and parameters: {'learning_rate': 4.923215705386883e-05, 'batch_size': 4, 'num_train_epochs': 9, 'weight_decay': 0.03513439983890576}. Best is trial 17 with value: 0.8122589802447074.
<ipython-input-10-9265609bdcc0>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-9265609bdcc0>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)
PyTorch: setting up devices
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: Th

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.417939,0.939011,0.828403,0.776224,0.799738
1,No log,0.440446,0.935165,0.809562,0.786713,0.792200
2,No log,0.425911,0.936813,0.815531,0.772727,0.790946
3,No log,0.423242,0.935165,0.813430,0.762238,0.784925
4,No log,0.406585,0.934615,0.804681,0.769231,0.785503
5,No log,0.408964,0.936264,0.808518,0.776224,0.790995
6,No log,0.410051,0.936264,0.806375,0.779720,0.791616
7,No log,0.409961,0.936813,0.807337,0.783217,0.794029


***** Running Evaluation *****
  Num examples = 260
  Batch size = 32
***** Running Evaluation *****
  Num examples = 260
  Batch size = 32
***** Running Evaluation *****
  Num examples = 260
  Batch size = 32
***** Running Evaluation *****
  Num examples = 260
  Batch size = 32
***** Running Evaluation *****
  Num examples = 260
  Batch size = 32
***** Running Evaluation *****
  Num examples = 260
  Batch size = 32
***** Running Evaluation *****
  Num examples = 260
  Batch size = 32
***** Running Evaluation *****
  Num examples = 260
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 260
  Batch size = 32


[I 2024-12-01 15:20:46,170] Trial 19 finished with value: 0.7940290034996405 and parameters: {'learning_rate': 1.5856472114281638e-05, 'batch_size': 16, 'num_train_epochs': 8, 'weight_decay': 0.007301952492841523}. Best is trial 17 with value: 0.8122589802447074.


Best trial:
FrozenTrial(number=17, state=TrialState.COMPLETE, values=[0.8122589802447074], datetime_start=datetime.datetime(2024, 12, 1, 15, 9, 4, 81983), datetime_complete=datetime.datetime(2024, 12, 1, 15, 13, 7, 988255), params={'learning_rate': 1.7658878046243477e-05, 'batch_size': 4, 'num_train_epochs': 8, 'weight_decay': 0.08094396293465898}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=5e-05, log=True, low=1e-06, step=None), 'batch_size': CategoricalDistribution(choices=(4, 8, 16)), 'num_train_epochs': IntDistribution(high=10, log=False, low=3, step=1), 'weight_decay': FloatDistribution(high=0.1, log=True, low=0.0001, step=None)}, trial_id=17, value=None)


In [12]:
best_hyperparams = study.best_trial.params

training_args = TrainingArguments(
    output_dir="./best_results",
    num_train_epochs=best_hyperparams["num_train_epochs"],
    per_device_train_batch_size=best_hyperparams["batch_size"],
    per_device_eval_batch_size=best_hyperparams["batch_size"] * 2,
    learning_rate=best_hyperparams["learning_rate"],
    weight_decay=best_hyperparams["weight_decay"],
    warmup_steps=10,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=2,
    report_to="none",
)

trainer = OneHotTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()

PyTorch: setting up devices
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1038
  Num Epochs = 8
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 2
  Total optimization steps = 1040


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.466324,0.940110,0.837726,0.765734,0.795706
2,No log,0.439595,0.940659,0.831631,0.779720,0.802298
3,No log,0.464817,0.936264,0.801466,0.790210,0.795126
4,0.001000,0.479064,0.934615,0.805460,0.790210,0.793693
5,0.001000,0.450835,0.939011,0.818867,0.786713,0.801087
6,0.001000,0.445890,0.938462,0.821997,0.776224,0.797833
7,0.001000,0.446255,0.937912,0.819800,0.776224,0.796857
8,0.000300,0.446388,0.938462,0.822422,0.776224,0.798192


***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
Saving model checkpoint to ./best_results/checkpoint-500
Configuration saved in ./best_results/checkpoint-500/config.json
Model weights saved in ./best_results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
***** Running Evaluation *****
  Num examples = 260
  Batch size = 8
Saving model checkpoint to ./best_results/checkpoint-1000
Configuration saved in ./best_results/checkpoint-1000/config.json
Model weights saved in ./best_results/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 260
  Batch size = 8


Training completed. Do not forget to share your model on 

TrainOutput(global_step=1040, training_loss=0.0006504525555870854, metrics={'train_runtime': 229.361, 'train_samples_per_second': 36.205, 'train_steps_per_second': 4.534, 'total_flos': 550053681979392.0, 'train_loss': 0.0006504525555870854, 'epoch': 8.0})

In [13]:
!mkdir 'best_model_distilbert_pharo'
!mkdir 'best_model_distilbert_pharo_tokenizer'

# Save model and tokenizer
trainer.save_model('./best_model_distilbert_pharo')
tokenizer.save_pretrained('./best_model_distilbert_pharo_tokenizer')

# Load model and tokenizer
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained('./best_model_distilbert_pharo')
tokenizer = DistilBertTokenizer.from_pretrained('./best_model_distilbert_pharo_tokenizer')

Saving model checkpoint to ./best_model_distilbert_pharo
Configuration saved in ./best_model_distilbert_pharo/config.json
Model weights saved in ./best_model_distilbert_pharo/pytorch_model.bin
tokenizer config file saved in ./best_model_distilbert_pharo_tokenizer/tokenizer_config.json
Special tokens file saved in ./best_model_distilbert_pharo_tokenizer/special_tokens_map.json
loading configuration file ./best_model_distilbert_pharo/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "

## Model Testing

In [14]:
X_test = list(test_df['combo'])
y_test = list(test_df['labels'])

In [15]:
df_test = pd.DataFrame({"combo":X_test,"labels":y_test})
test_text = df_test.combo.values
test_label = df_test.labels.values

print(test_label)

[array([0, 0, 1, 1, 0, 0, 0]) array([0, 1, 0, 0, 0, 0, 0])
 array([0, 0, 1, 0, 0, 0, 0]) array([0, 0, 1, 0, 0, 0, 0])
 array([0, 0, 1, 0, 0, 0, 0]) array([0, 0, 1, 0, 0, 0, 0])
 array([0, 0, 1, 0, 0, 0, 0]) array([0, 0, 1, 0, 0, 0, 0])
 array([0, 1, 0, 0, 0, 0, 0]) array([0, 0, 0, 0, 1, 0, 0])
 array([0, 0, 0, 0, 1, 0, 0]) array([0, 0, 1, 0, 0, 0, 0])
 array([0, 0, 1, 0, 0, 0, 0]) array([0, 1, 0, 0, 0, 0, 0])
 array([0, 1, 0, 0, 0, 0, 0]) array([0, 1, 0, 0, 0, 0, 0])
 array([0, 1, 0, 0, 0, 0, 0]) array([0, 1, 0, 0, 0, 0, 0])
 array([0, 1, 0, 0, 0, 0, 0]) array([1, 0, 0, 0, 0, 0, 0])
 array([1, 0, 0, 0, 0, 0, 0]) array([1, 0, 0, 0, 0, 0, 0])
 array([1, 0, 0, 0, 0, 0, 0]) array([0, 0, 0, 0, 1, 0, 0])
 array([1, 0, 0, 0, 0, 0, 0]) array([0, 0, 0, 0, 0, 0, 1])
 array([0, 0, 0, 0, 0, 1, 0]) array([0, 0, 0, 0, 0, 1, 0])
 array([0, 0, 0, 0, 0, 1, 0]) array([1, 0, 0, 0, 0, 0, 0])
 array([0, 0, 0, 0, 1, 0, 0]) array([0, 0, 1, 0, 0, 0, 0])
 array([0, 0, 1, 0, 0, 0, 0]) array([0, 0, 1, 0, 0, 0, 0

In [16]:
type(test_label[0])

numpy.ndarray

In [17]:
test_dataset = TextClassificationDataset(test_text, test_label, tokenizer)

# Measure inference time
start_time = time.time()

# Predict on the test dataset
predictions = trainer.predict(test_dataset)

# Calculate elapsed time
end_time = time.time()
inference_time = end_time - start_time

# Compute metrics using the `compute_metrics` function
metrics = compute_metrics(predictions)

# Display metrics and inference time
print("Evaluation Metrics:", metrics)
print(f"Inference Time: {inference_time:.2f} seconds")

***** Running Prediction *****
  Num examples = 289
  Batch size = 8


Evaluation Metrics: {'accuracy': 0.9263470093919921, 'precision': 0.7581272892738996, 'recall': 0.7508305647840532, 'f1': 0.753951274874274}
Inference Time: 2.42 seconds


In [18]:
!zip -r best_model_distilbert_pharo.zip './best_model_distilbert_pharo'
!zip -r best_model_distilbert_pharo_tokenizer.zip './best_model_distilbert_pharo_tokenizer'

# For downloading model locally
# from google.colab import files
# files.download('checkpoint-6000.zip')

  adding: best_model_distilbert_pharo/ (stored 0%)
  adding: best_model_distilbert_pharo/pytorch_model.bin (deflated 8%)
  adding: best_model_distilbert_pharo/training_args.bin (deflated 51%)
  adding: best_model_distilbert_pharo/config.json (deflated 53%)
  adding: best_model_distilbert_pharo_tokenizer/ (stored 0%)
  adding: best_model_distilbert_pharo_tokenizer/tokenizer_config.json (deflated 43%)
  adding: best_model_distilbert_pharo_tokenizer/special_tokens_map.json (deflated 40%)
  adding: best_model_distilbert_pharo_tokenizer/vocab.txt (deflated 53%)


In [20]:
# prompt: transfer the checkpoint-6000.zip created to the google drive

from google.colab import drive
drive.mount('/content/drive')

!cp best_model_distilbert_pharo.zip "/content/drive/MyDrive/FYP/Revised/Models/DistilBert_pharo/"
!cp best_model_distilbert_pharo_tokenizer.zip "/content/drive/MyDrive/FYP/Revised/Models/DistilBert_pharo_Tokenizer/"

Mounted at /content/drive


In [ ]:
# Save the tokenizer
# tokenizer.save_pretrained('/content/drive/MyDrive/FYP/Revised/Models/DistilBert_Python_Tokenizer/')

# **Load and Test Model**

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Replace 'path/to/checkpoint-folder' with the actual path to your checkpoint folder.
model_folder = '/content/drive/MyDrive/FYP/Revised/Models/DistilBert_pharo/'
tokenizer_folder = '/content/drive/MyDrive/FYP/Revised/Models/DistilBert_pharo_Tokenizer/'

!unzip '/content/drive/MyDrive/FYP/Revised/Models/DistilBert_pharo/best_model_distilbert_pharo.zip' -d './'
!unzip '/content/drive/MyDrive/FYP/Revised/Models/DistilBert_pharo_Tokenizer/best_model_distilbert_pharo_tokenizer.zip' -d './'

Archive:  /content/drive/MyDrive/FYP/Revised/Models/DistilBert_pharo/best_model_distilbert_pharo.zip
replace ./best_model_distilbert_pharo/pytorch_model.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./best_model_distilbert_pharo/pytorch_model.bin  
  inflating: ./best_model_distilbert_pharo/training_args.bin  
  inflating: ./best_model_distilbert_pharo/config.json  
Archive:  /content/drive/MyDrive/FYP/Revised/Models/DistilBert_pharo_Tokenizer/best_model_distilbert_pharo_tokenizer.zip
replace ./best_model_distilbert_pharo_tokenizer/tokenizer_config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./best_model_distilbert_pharo_tokenizer/tokenizer_config.json  
  inflating: ./best_model_distilbert_pharo_tokenizer/special_tokens_map.json  
  inflating: ./best_model_distilbert_pharo_tokenizer/vocab.txt  


In [23]:
best_model_distilbert_pharo = 'best_model_distilbert_pharo'
best_model_distilbert_pharo_tokenizer = 'best_model_distilbert_pharo_tokenizer'

In [27]:
# Load the model and tokenizer from the checkpoint
model = DistilBertForSequenceClassification.from_pretrained(best_model_distilbert_pharo)
tokenizer = DistilBertTokenizer.from_pretrained(best_model_distilbert_pharo_tokenizer)

text = "relative anchor takes an arbitrary element as"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
output

loading configuration file best_model_distilbert_pharo/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "multi_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.17.0",
  "voca

SequenceClassifierOutput(loss=None, logits=tensor([[ -7.6903, -11.9937,   8.0281, -14.3547, -13.3136, -12.5140, -10.8436]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [28]:
logits = output.logits

# Apply sigmoid to convert logits to probabilities
probabilities = torch.sigmoid(logits)

# Define a threshold to determine if a class is positive
threshold = 0.5
predicted_indices = (probabilities > threshold).nonzero(as_tuple=True)[1]

# Convert indices to a list for output
predicted_classes = (predicted_indices + 1).tolist()

# Format the output to display classes
if len(predicted_classes) > 0:
    predicted_classes_str = ", ".join(map(str, predicted_classes))
    print(f"Predicted class/es: {predicted_classes_str}")
else:
    print("No positive classes predicted.")

Predicted class/es: 3


In [26]:
# import torch

# # Logits from the model output
# logits =  output.logits #torch.tensor([[-17.5500, -13.8129, 9.7042, -16.6331, -6.6066]])

# # Get the index of the highest score
# predicted_class = ((torch.argmax(logits, dim=1).item()) + 1)

# print(f"Predicted class: {predicted_class}")
